In [ ]:
rm -r /content/face_and_emotion_detection

Clone the github **repository**

In [1]:
!git clone https://github.com/KhurramShahzadODM/face_and_emotion_detection.git

Cloning into 'face_and_emotion_detection'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 57 (delta 10), reused 48 (delta 5), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [2]:
!pip install face_recognition


     |████████████████████████████████| 100.1 MB 22 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=f2d39d3d02e4d05972aead280a2c20b3f370490065a844372b60a8f784667ab8
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
import shutil
import os
# clean and rebuild the image folders
input_folder = '/content/face_and_emotion_detection/inputs'
if os.path.exists(input_folder):
  shutil.rmtree(input_folder)
os.makedirs(input_folder)

output_folder = '/content/face_and_emotion_detection/outputs'
if os.path.exists(output_folder):
  shutil.rmtree(output_folder)
os.makedirs(output_folder)

# upload images (PNG or JPG)
image_names = list(files.upload().keys())
for image_name in image_names:
  shutil.move(image_name, os.path.join(input_folder, image_name))


Saving trump_speech_short.mp4 to trump_speech_short.mp4


In [2]:
from imutils import paths
import os
import cv2
import numpy as np
from google.colab import files
import numpy as np
import face_recognition
from keras.models import load_model
from google.colab.patches import cv2_imshow
import imutils
import keras


imagePaths = list(paths.list_images('/content/face_and_emotion_detection/trump'))
# initialize the list of known encodings and known names
knownEncodings = []
knownNames = []
for (i, imagePath) in enumerate(imagePaths):
  # extract the person name from the image path
  print("[INFO] processing image {}/{}".format(i + 1,
    len(imagePaths)))
  name = imagePath.split(os.path.sep)[-2]
  # load the input image and convert it from BGR (OpenCV ordering)
  # to dlib ordering (RGB)
  image = cv2.imread(imagePath)
  rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # rgb = cv2.resize(rgb, (512,640))
  rgb = imutils.resize(rgb, width=750)
  boxes = face_recognition.face_locations(rgb,model="cnn")
  encodings = face_recognition.face_encodings(rgb, boxes)
  for encoding in encodings:
    # add each encoding + name to our set of known names and
    # encodings
    knownEncodings.append(encoding)
    knownNames.append(name)

[INFO] processing image 1/11
[INFO] processing image 2/11
[INFO] processing image 3/11
[INFO] processing image 4/11
[INFO] processing image 5/11
[INFO] processing image 6/11
[INFO] processing image 7/11
[INFO] processing image 8/11
[INFO] processing image 9/11
[INFO] processing image 10/11
[INFO] processing image 11/11


In [9]:
# from keras.models import load_model
# import cv2
# import numpy as np
# rows, cols = (7, 1)
# results = [[0]*cols]*rows
results = [[] for i in range(7)]
print(results)
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
video_path = "/content/face_and_emotion_detection/inputs/trump_1min_sec.mp4"
# video_path = "/content/face_and_emotion_detection/inputs/trump-1mint.mp4"

File_name = os.path.basename(video_path)

# cap = cv2.VideoCapture(video_path)
emotion_dict= {'Angry': 0, 'Sad': 5, 'Neutral': 4, 'Disgust': 1, 'Surprise': 6, 'Fear': 2, 'Happy': 3}

total_frame_wd_face = 0

cap = cv2.VideoCapture(video_path)

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
# img = cv2.imread('/content/face_and_emotion_detection/inputs/trump.jpg', cv2.IMREAD_COLOR)

# cv2_imshow(img)
# Read until video is completed
# while(cap.isOpened()):
while(total_frame_wd_face < 1):
  # print(total_frame_wd_face)
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    # Display the resulting frame
    # cv2_imshow(frame)
    # frame = img
    # frame = cv2.resize(frame, (512,640))
    frame = imutils.resize(frame, width=750)

    faces = face_recognition.face_locations(frame,model="cnn")
    # print(len(faces))
    # plt.imshow(face_image1)
    trump_face = 0
    for face in faces:
      # print(face)
      encoding_2 = face_recognition.face_encodings(frame, [face])
      matches = face_recognition.compare_faces(knownEncodings,encoding)
      # print(matches)
      # print(len(knownEncodings))
      # print(sum(matches))
      if sum(matches) > 0:
        trump_face = face
        break
    # print(trump_face)
    if(trump_face != 0):
      total_frame_wd_face += 1
      top, right, bottom, left = trump_face
      face_image = frame[top:bottom, left:right]
      face_image = cv2.resize(face_image, (48,48))
      # cv2_imshow(face_image)
      face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
      face_image = np.reshape(face_image, [1, face_image.shape[0], face_image.shape[1], 1])
     
      # y_1 = model_1.predict(face_image)

      # l = np.argmax(y_1, axis=1)
      # print('conf : '+ str(format(y_1[0][l[0]],'f')))
      # print('label : ' + str(emotions[l[0]][0]))
      
      model = load_model("/content/face_and_emotion_detection/model/model_v6_23.hdf5")
      result = model.predict(face_image)
      # print(result)
      predicted_class = np.argmax(result)
      # label_map = dict((v,k) for k,v in emotion_dict.items())
      # print(label_map[predicted_class])
      # print(predicted_class)
      confidence = format(result[0][predicted_class],'f')
      # print(confidence)
      # print(predicted_class)
      results[predicted_class].append(confidence)
      # print(predicted_label)


      # results = face_recognition.compare_faces(knownEncodings, encoding_2,tolerance=0.5)
      # print(results)
      # if(results[0]):
      #   print('trump is in the frame')
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()
label_map = dict((v,k) for k,v in emotion_dict.items())
# for i,result in results:
f = open("/content/face_and_emotion_detection/results.txt", "a")
File_name
f.write(str(File_name + "\n"))
f.write("label, Percent \n")

print ("label, Percent")
for idx, result in enumerate(results):
    # print(result)
    result = np.array(result).astype(np.float)
    print(label_map[idx], "  " , int(np.sum(result))/total_frame_wd_face*100)
    f.write(str(label_map[idx] + "  " + str(int(np.sum(result))/total_frame_wd_face*100)) + "\n")
f.write("\n")
f.close()
  # predicted_label = label_map[predicted_class]

[[], [], [], [], [], [], []]
label, Percent
Angry    0.0
Disgust    0.0
Fear    100.0
Happy    0.0
Neutral    0.0
Sad    0.0
Surprise    0.0


In [11]:
a_file = open("/content/face_and_emotion_detection/results.txt")
lines = a_file. readlines()
for line in lines:
  print(line)

trump_1min_sec.mp4

label, Percent 

Angry  0.0

Disgust  0.0

Fear  100.0

Happy  0.0

Neutral  0.0

Sad  0.0

Surprise  0.0



trump_1min_sec.mp4

label, Percent 

Angry  0.0

Disgust  0.0

Fear  100.0

Happy  0.0

Neutral  0.0

Sad  0.0

Surprise  0.0





In [ ]:
!zip /content/face.zip /content/face_and_emotion_detection/*

  adding: content/face_and_emotion_detection/inputs/ (stored 0%)
  adding: content/face_and_emotion_detection/LICENSE (deflated 41%)
  adding: content/face_and_emotion_detection/model/ (stored 0%)
  adding: content/face_and_emotion_detection/outputs/ (stored 0%)
  adding: content/face_and_emotion_detection/README.md (deflated 4%)
  adding: content/face_and_emotion_detection/test_images/ (stored 0%)
  adding: content/face_and_emotion_detection/trump/ (stored 0%)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

emotions = {
    0: ['Angry', (0,0,255), (255,255,255)],
    1: ['Disgust', (0,102,0), (255,255,255)],
    2: ['Fear', (255,255,153), (0,51,51)],
    3: ['Happy', (153,0,153), (255,255,255)],
    4: ['Sad', (255,0,0), (255,255,255)],
    5: ['Surprise', (0,255,0), (255,255,255)],
    6: ['Neutral', (160,160,160), (255,255,255)]
}
num_classes = len(emotions)
input_shape = (48, 48, 1)
weights_1 = '/content/face_and_emotion_detection/model/vggnet.h5'

In [ ]:
class VGGNet(Sequential):
    def __init__(self, input_shape, num_classes, checkpoint_path, lr=1e-3):
        super().__init__()
        self.add(Rescaling(1./255, input_shape=input_shape))
        self.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'))
        self.add(BatchNormalization())
        self.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.5))

        self.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.4))

        self.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.5))

        self.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.4))

        self.add(Flatten())
        
        self.add(Dense(1024, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(256, activation='relu'))

        self.add(Dense(num_classes, activation='softmax'))

        self.compile(optimizer=Adam(learning_rate=lr),
                    loss=categorical_crossentropy,
                    metrics=['accuracy'])
        
        self.checkpoint_path = checkpoint_path
model_1 = VGGNet(input_shape, num_classes, weights_1)
model_1.load_weights(model_1.checkpoint_path)